In [1]:
import socket
import ssl
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.serialization import load_pem_private_key, load_pem_public_key
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.serialization import load_pem_private_key
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives import serialization
import os


In [2]:

def generate_rsa_keypair():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    return private_key

def generate_dh_keypair():
    parameters = dh.generate_parameters(generator=2, key_size=2048)
    private_key = parameters.generate_private_key()
    public_key = private_key.public_key()
    return private_key, public_key

def derive_dh_shared_key(private_key, public_key):
    shared_key = private_key.exchange(public_key)
    return shared_key

def rsa_encrypt(public_key, plaintext):
    ciphertext = public_key.encrypt(
        plaintext.encode(),
        padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None)
        )
    return ciphertext

def rsa_decrypt(private_key, ciphertext):
    plaintext = private_key.decrypt(
    ciphertext,
    padding.OAEP( mgf=padding.MGF1(algorithm=hashes.SHA256()),
    algorithm=hashes.SHA256(),
    label=None))
    return plaintext.decode()

def aes_encrypt(key, plaintext):
    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(plaintext.encode()) + encryptor.finalize()
    return iv + ciphertext

def aes_decrypt(key, ciphertext):
    iv = ciphertext[:16]
    ciphertext = ciphertext[16:]
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    return plaintext.decode()

def start_client():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)
    context.load_verify_locations(cafile="server.crt")
    context.check_hostname = False
    client_socket.connect(("localhost", 8443))
    print("Connected to server.")
    
    with context.wrap_socket(client_socket, server_hostname="localhost") as ssock:
        
        print("SSL/TLS connection established.")

        encryption_choice = input("Please choose encryption method: 1 for RSA or 2 for AES (Diffie-Hellman)")
        # ssock.send(.encode())
        ssock.send(encryption_choice.encode())
        # encryption_choice = ssock.recv(1024).decode()
        # print(f"Encryption choice: {encryption_choice}")
        
        if encryption_choice == "1":
            # RSA encryption and PKI key exchange
            private_key = generate_rsa_keypair()
            public_key = private_key.public_key()
            ssock.send(public_key.public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo
            ))
            
            #receive public key from server
            received_public_key = ssock.recv(2048)
            server_public_key = load_pem_public_key(received_public_key, backend=default_backend())
            
            #send msg to server
            msg = input("Enter your message: ")
            ciphertext = rsa_encrypt(server_public_key, msg)
            print(f"Encrypted RSA message: {ciphertext}")
            ssock.send(ciphertext)
            print("Message sent.")
            return
            data = ssock.recv(2048)
            
            message = rsa_decrypt(private_key, data)
            print(f"Decrypted RSA message: {message}")

        elif encryption_choice == "2":
            # AES encryption and Diffie-Hellman key exchange
            bob_private_key, bob_public_key = generate_dh_keypair()
            ssock.send(bob_public_key.public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo
            ))

            received_public_key = ssock.recv(2048)
            received_public_key = serialization.load_pem_public_key(received_public_key, backend=default_backend())
            shared_key = derive_dh_shared_key(bob_private_key, received_public_key)

            data = ssock.recv(1024)
            message = aes_decrypt(shared_key, data)
            print(f"Decrypted AES message: {message}")
    



In [3]:
if __name__ == "__main__":
    start_client()
#openssl req -x509 -newkey rsa:4096 -keyout ca.key -out ca.crt -days 365
    """
    info for ca.crt
    PEM pass phrase: 03455
    country name: PK
    state or province name: Punjab
    locality name: Rawalpindi
    organization name: FAST
    organizational unit name: Computing
    common name: FARQ
    email address: i200621@nu.edu.pk
    """



Connected to server.
SSL/TLS connection established.


Encrypted RSA message: b'\x01\x0c\xc9\x92\x14\xeav7\xf0\x85\x9er\x07\xcf\r`-y\x9f\xc3Y\x02a\x98k\xd3\xd4\x85\x90\xb3\xb2\xc3\xfb\xc12\x891\xa9\x15\xed,\xa9jD\xd5C-\x9dA\xebi\x97\x15D=\xd9\x1a\x1f\xdb\x9c\xd4\xeeV\x0c\x07\xad\x8eA\x82\xae\xe6\x1f\xb7C,J~\xe6\xf5\xd3\xb6\xde\xcb\n\x03\xb8\n-\xfe\x80\xe9|\xd1\x8c\xfc-\x16\x08LU\x08\xa2\x8b\xa3\xafA\x9f\x95\xc5M[\x0e_e\xfcB\xb0\xf9\xfc\xdc\x85BS\xf7\x8bJ\xdb~>\x1e\xab\xbd-b\xee\xe4^\xb8M\xcc\xbf\x87\x88QB\xe3\xd6\x86h\x0b\xdfgFu\x8a\xf4l\xa1\xb0\x07\xcb\xe4\xc7\x16\xaer\x11\x90\x88\xd4\xd9DHA\x1a\x86\x19\xc9$\xa1\x95LK\xa0\x05\xf1\xde!\xd1\xeb\xdc\x81D-\xa8\xda\x18\x13F\xd8\xc1\xb6a4\xb9\x9a\xe3\x98\x06\x7f\xbbG\x12E\xdf\x99.\xa6+\x13\x88\x85\x15\xa5\xe4\xd3\x11\x1e5\xa1\xa9\xe2O:\xaa\rh\xa2\xbaH\x91~OP\xba\x8aa\xbf{i\xd8\xb7\xde\xaeJ\x05'
Message sent.
